# CS 5588 — Week 2 Hands-On: Applied RAG for Product & Venture Development (Two-Step)
**Initiation (20 min, Jan 27)** → **Completion (60 min, Jan 29)**

**Submission:** Survey + GitHub  
**Due:** **Jan 29 (Thu), end of class**

## New Requirement (Important)
For **full credit (2% individual)** you must:
1) Use **your own project-aligned dataset** (not only benchmark)  
2) Add **your own explanations** for key steps

### ✅ “Cell Description” rule (same style as CS 5542)
After each **IMPORTANT** code cell, add a short Markdown **Cell Description** (2–5 sentences):
- What the cell does
- Why it matters for a **product-grade** RAG system
- Any design choices (chunk size, α, reranker, etc.)

> Treat these descriptions as **mini system documentation** (engineering + product thinking).


## Project Dataset Guide (Required for Full Credit)

### Minimum requirements
- **5–25 documents** (start small; scale later)
- Prefer **plain text** documents (`.txt`)
- Put files in a folder named: `project_data/`

### Recommended dataset types (choose one)
- Policies / guidelines / compliance docs
- Technical docs / manuals / SOPs
- Customer support FAQs / tickets (de-identified)
- Research notes / literature summaries
- Domain corpus (healthcare, cybersecurity, business, etc.)

> Benchmarks are optional, but **cannot** earn full credit by themselves.


## 0) One-Click Setup + Import Check  ✅ **IMPORTANT: Add Cell Description after running**
If you are in **Google Colab**, run the install cell below, then **Runtime → Restart session** if imports fail.


In [46]:
# CS 5588 Lab 2 — One-click dependency install (Colab)
!pip -q install -U sentence-transformers chromadb faiss-cpu scikit-learn rank-bm25 transformers accelerate

import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
print("✅ If imports fail later: Runtime → Restart session and run again.")

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
✅ If imports fail later: Runtime → Restart session and run again.


### ✍️ Cell Description (Student)
Write 2–5 sentences explaining what the setup cell does and why restarting the runtime sometimes matters after pip installs.

This cell installs the libraries needed to build a RAG pipeline, including embeddings, a vector store, sparse retrieval, and evaluation tools. It also prints the Python and platform version so the environment is reproducible and easier to debug.

Restarting the runtime after pip install matters because Colab/Jupyter sometimes keeps older versions of packages loaded in memory, which can cause import errors or version conflicts.

# STEP 1 — INITIATION (Jan 27, 20 minutes)
**Goal:** Define the **product**, **users**, **dataset reality**, and **trust risks**.

> This is a **product milestone**, not a coding demo.


## 1A) Product Framing (Required)  ✅ **IMPORTANT: Add Cell Description after running**
Fill in the template below like a founder/product lead.


In [47]:
product = {
  "product_name": "SecOps RAG Assistant — Incident Response Copilot",
  "target_users": "SOC analysts, junior security engineers, and IT responders who need fast, consistent guidance during phishing incidents, suspected breaches, and post-incident review.",
  "core_problem": "During an incident, responders waste time searching scattered guidance and may miss critical steps. Incidents can disrupt business operations and information security, so teams need quick, predictable, evidence-backed answers under pressure.",
  "why_rag_not_chatbot": "A generic chatbot can sound confident but hallucinate security steps or definitions. RAG is needed so answers are grounded in the team’s approved knowledge base (incident management, incident response plans, breach definitions, and phishing concepts) and can provide citations.",
  "failure_harms_who_and_how": "Wrong guidance can delay containment/eradication, increase downtime, and worsen breach impact. It can lead to exposure of personal information and legal/compliance risk, harming customers and the organization’s reputation, while also causing responders to make high-stakes decisions with false confidence.",
}
product

{'product_name': 'SecOps RAG Assistant — Incident Response Copilot',
 'target_users': 'SOC analysts, junior security engineers, and IT responders who need fast, consistent guidance during phishing incidents, suspected breaches, and post-incident review.',
 'core_problem': 'During an incident, responders waste time searching scattered guidance and may miss critical steps. Incidents can disrupt business operations and information security, so teams need quick, predictable, evidence-backed answers under pressure.',
 'why_rag_not_chatbot': 'A generic chatbot can sound confident but hallucinate security steps or definitions. RAG is needed so answers are grounded in the team’s approved knowledge base (incident management, incident response plans, breach definitions, and phishing concepts) and can provide citations.',
 'failure_harms_who_and_how': 'Wrong guidance can delay containment/eradication, increase downtime, and worsen breach impact. It can lead to exposure of personal information and

### ✍️ Cell Description (Student)
Explain your product in 3–5 sentences: who the user is, what pain point exists today, and why grounded RAG helps.

This cell defines my RAG product. It targets SOC/IT responders who need fast, reliable guidance during phishing and breach-related incidents. The main pain point is that incident response knowledge is scattered across documents, and delays or wrong steps can disrupt operations and increase harm. Grounded RAG helps because it forces the system to answer using retrieved evidence from the incident management + data breach + phishing + information security documents, instead of guessing. In a high-stakes security setting, this reduces hallucinations and supports trust through citations and predictable, policy-aligned responses.


## 1B) Dataset Reality Plan (Required)  ✅ **IMPORTANT: Add Cell Description after running**
Describe where your data comes from **in the real world**.


In [48]:
dataset_plan = {
  "data_owner": "Public (Wikipedia) for this prototype; in production it would be owned by the organization’s security/GRC and IT teams (internal knowledge base + IR playbooks).",
  "data_sensitivity": "Public for this lab dataset; in production: internal/confidential because incident procedures, internal tooling, and post-incident notes may expose sensitive operational details.",
  "document_types": "Security fundamentals reference docs (information security concepts), incident management/incident response guidance, breach definitions, phishing descriptions, and incident management process overviews.",
  "expected_scale_in_production": "Prototype: 5–25 docs. Production: 200–2,000 docs (policies, IR playbooks, runbooks, FAQs, and de-identified ticket/incident summaries).",
  "data_reality_check_paragraph": (
    "For this lab, I’m using a small public corpus converted to text files. "
    "In a real company deployment, the RAG corpus would come from internal sources like security policies, incident response playbooks, "
    "SOC runbooks, and de-identified incident tickets. A key reality constraint is that true incident documentation can include sensitive data "
    "(customer information, credentials, IPs, vulnerabilities), so ingestion would require access controls, redaction/de-identification, and retention rules. "
    "The system should also log citations and restrict answers when evidence is missing to avoid unsafe or non-compliant guidance."
  ),
}
dataset_plan

{'data_owner': 'Public (Wikipedia) for this prototype; in production it would be owned by the organization’s security/GRC and IT teams (internal knowledge base + IR playbooks).',
 'data_sensitivity': 'Public for this lab dataset; in production: internal/confidential because incident procedures, internal tooling, and post-incident notes may expose sensitive operational details.',
 'document_types': 'Security fundamentals reference docs (information security concepts), incident management/incident response guidance, breach definitions, phishing descriptions, and incident management process overviews.',
 'expected_scale_in_production': 'Prototype: 5–25 docs. Production: 200–2,000 docs (policies, IR playbooks, runbooks, FAQs, and de-identified ticket/incident summaries).',
 'data_reality_check_paragraph': 'For this lab, I’m using a small public corpus converted to text files. In a real company deployment, the RAG corpus would come from internal sources like security policies, incident resp

### ✍️ Cell Description (Student)
Write 2–5 sentences describing where this data would come from in a real deployment and any privacy/regulatory constraints.

This cell documents the data reality behind the RAG system: where the corpus would come from and what constraints apply in production. Although my current dataset is public Wikipedia text, a real deployment would rely on internal SOC runbooks, incident response playbooks, and de-identified tickets, which are often confidential. That means we’d need privacy protections like redaction, access controls, and retention/governance rules to avoid leaking sensitive operational details. Defining this upfront matters because it shapes how we ingest documents, secure retrieval, and decide when the assistant must say when there is not enough evidence.

## 1C) User Stories + Mini Rubric (Required)  ✅ **IMPORTANT: Add Cell Description after running**
Define **3 user stories** (U1 normal, U2 high-stakes, U3 ambiguous/failure) + rubric for evidence and correctness.


In [49]:
user_stories = {
  "U1_normal": {
    "user_story": "As a SOC analyst, I want a quick definition of phishing and what to look for in a suspicious email so that I can triage alerts faster and avoid clicking risky links.",
    "acceptable_evidence": [
      "Phishing.txt (definition of phishing + typical components/signals of phishing emails)",
      "Information_security.txt (why protecting confidentiality matters when credentials are targeted)"
    ],
    "correct_answer_must_include": [
      "Define phishing as a social engineering scam to trick users into revealing sensitive information and/or installing malware",
      "List at least 3 common phishing indicators (e.g., urgent tone, fake link, spoofed/similar domain, generic greeting, branding errors)"
    ],
  },

  "U2_high_stakes": {
    "user_story": "As an incident response lead, I want guidance on what actions to take after discovering a potential data breach so that I can contain impact, investigate scope, and meet notification/legal obligations.",
    "acceptable_evidence": [
      "Data_breach.txt (what a data breach is + common post-breach efforts like containment, investigation, notifications)",
      "Computer_security_incident_management.txt (incident response plan + legal/compliance implications)",
      "Incident_management.txt (goal of restoring normal operations and minimizing business impact)"
    ],
    "correct_answer_must_include": [
      "State what a data breach is (unauthorized exposure/disclosure/loss of personal information or unauthorized access)",
      "Include post-breach actions: contain the breach, investigate scope/cause, and notify affected people as required by law",
      "Explicitly mention legal/compliance risk and that response may require non-IT roles (e.g., legal) in the plan"
    ],
  },

  "U3_ambiguous_failure": {
    "user_story": "As a compliance-minded responder, I want to know exactly what breach notification deadline applies in my state so that I can file notifications correctly and avoid penalties.",
    "acceptable_evidence": [
      "Data_breach.txt (general statement that notification is required by law in many jurisdictions, but not specific deadlines)",
      "Computer_security_incident_management.txt (legal implications exist, but no jurisdiction-specific timelines)"
    ],
    "correct_answer_must_include": [
      "The system must say 'not enough evidence' because the dataset does not include state-specific notification deadlines",
      "It must cite the retrieved evidence showing only general legal/notification mentions and recommend consulting official legal/regulatory sources or internal counsel",
      "Ask a clarifying question (jurisdiction + org policy) instead of guessing"
    ],
  },
}
user_stories

{'U1_normal': {'user_story': 'As a SOC analyst, I want a quick definition of phishing and what to look for in a suspicious email so that I can triage alerts faster and avoid clicking risky links.',
  'acceptable_evidence': ['Phishing.txt (definition of phishing + typical components/signals of phishing emails)',
   'Information_security.txt (why protecting confidentiality matters when credentials are targeted)'],
  'correct_answer_must_include': ['Define phishing as a social engineering scam to trick users into revealing sensitive information and/or installing malware',
   'List at least 3 common phishing indicators (e.g., urgent tone, fake link, spoofed/similar domain, generic greeting, branding errors)']},
 'U2_high_stakes': {'user_story': 'As an incident response lead, I want guidance on what actions to take after discovering a potential data breach so that I can contain impact, investigate scope, and meet notification/legal obligations.',
  'acceptable_evidence': ['Data_breach.txt (

### ✍️ Cell Description (Student)
Explain why U2 is “high-stakes” and what the system must do to avoid harm (abstain, cite evidence, etc.).

U2 is high-stakes because breach response impacts customers, business continuity, and legal compliance—wrong guidance can increase harm or cause regulatory violations. For this reason, the system must cite evidence from the breach and incident-management documents and avoid confident guessing. If the retrieved evidence is incomplete, the assistant should abstain (not enough evidence) and ask clarifying questions rather than inventing notification rules or procedures. This rubric forces the RAG system to prioritize trustworthy, evidence-grounded guidance over fluent but risky answers.


## 1D) Trust & Risk Table (Required)
Fill at least **3 rows**. These risks should match your product and user stories.


In [50]:
risk_table = [
  {
    "risk": "Hallucination",
    "example_failure": "The assistant invents a specific breach notification deadline (e.g., '72 hours in every state') even though our dataset only mentions notification in general.",
    "real_world_consequence": "Missed legal deadlines or incorrect reporting → regulatory penalties, lawsuits, reputational damage, and increased customer harm.",
    "safeguard_idea": "Force citations + abstain"
  },
  {
    "risk": "Omission",
    "example_failure": "For a breach response question, the assistant gives a definition but fails to mention containment/investigation steps or the need to involve legal/compliance teams.",
    "real_world_consequence": "Delayed containment and incomplete response → greater breach impact, longer downtime, and non-compliance due to missing required actions.",
    "safeguard_idea": "Recall tuning + hybrid retrieval"
  },
  {
    "risk": "Bias/Misleading",
    "example_failure": "The assistant over-simplifies phishing by implying only obvious scam emails are phishing, causing users to underestimate spear-phishing or subtle attacks.",
    "real_world_consequence": "Lower vigilance → higher click-through rates, credential theft, and successful compromises.",
    "safeguard_idea": "Reranking rules + human review"
  },
]
risk_table

[{'risk': 'Hallucination',
  'example_failure': "The assistant invents a specific breach notification deadline (e.g., '72 hours in every state') even though our dataset only mentions notification in general.",
  'real_world_consequence': 'Missed legal deadlines or incorrect reporting → regulatory penalties, lawsuits, reputational damage, and increased customer harm.',
  'safeguard_idea': 'Force citations + abstain'},
 {'risk': 'Omission',
  'example_failure': 'For a breach response question, the assistant gives a definition but fails to mention containment/investigation steps or the need to involve legal/compliance teams.',
  'real_world_consequence': 'Delayed containment and incomplete response → greater breach impact, longer downtime, and non-compliance due to missing required actions.',
  'safeguard_idea': 'Recall tuning + hybrid retrieval'},
 {'risk': 'Bias/Misleading',
  'example_failure': 'The assistant over-simplifies phishing by implying only obvious scam emails are phishing, c

✅ **Step 1 Checkpoint (End of Jan 27)**
Commit (or submit) your filled templates:
- `product`, `dataset_plan`, `user_stories`, `risk_table`


# STEP 2 — COMPLETION (Jan 29, 60 minutes)
**Goal:** Build a working **product-grade** RAG pipeline:
Chunking → Keyword + Vector Retrieval → Hybrid α → Governance Rerank → Grounded Answer → Evaluation


## 2A) Project Dataset Setup (Required for Full Credit)  ✅ **IMPORTANT: Add Cell Description after running**

### Colab Upload Tips
- Left sidebar → **Files** → Upload `.txt`
- Place them into `project_data/`

This cell creates the folder and shows how many files were found.


In [51]:
import os, glob, shutil
from pathlib import Path

PROJECT_FOLDER = "project_data"
os.makedirs(PROJECT_FOLDER, exist_ok=True)

# (Optional helper) Move any .txt in current directory into project_data/
moved = 0
for fp in glob.glob("*.txt"):
    shutil.move(fp, os.path.join(PROJECT_FOLDER, os.path.basename(fp)))
    moved += 1

files = sorted(glob.glob(os.path.join(PROJECT_FOLDER, "*.txt")))
print("✅ project_data/ ready | moved:", moved, "| files:", len(files))
print("Example files:", files[:5])

✅ project_data/ ready | moved: 0 | files: 5
Example files: ['project_data/Computer_security_incident_management.txt', 'project_data/Data_breach.txt', 'project_data/Incident_management.txt', 'project_data/Information_security.txt', 'project_data/Phishing.txt']


### ✍️ Cell Description (Student)
List what dataset you used, how many docs, and why they reflect your product scenario (not just a toy example).

For this project, I’m using 5 text documents cybersecurity corpus which are Incident management, Computer security incident management, Data breach, Phishing, and Information security. These documents reflect my product scenario because a SecOps RAG assistant needs grounded definitions and procedural context for common incident-response questions (triage, breach impact, phishing recognition, and security fundamentals).

Even though the sources are public, the structure mirrors a real deployment where the same pipeline would ingest internal playbooks, policies, and runbooks. Starting with a small but coherent set also makes it easier to evaluate retrieval quality and identify failures before scaling to a larger enterprise knowledge base.


## 2B) Load Documents + Build Chunks  ✅ **IMPORTANT: Add Cell Description after running**
This milestone cell loads `.txt` documents and produces chunks using either **fixed** or **semantic** chunking.


In [52]:
import re

def load_project_docs(folder="project_data", max_docs=25):
    paths = sorted(Path(folder).glob("*.txt"))[:max_docs]
    docs = []
    for p in paths:
        txt = p.read_text(encoding="utf-8", errors="ignore").strip()
        if txt:
            docs.append({"doc_id": p.name, "text": txt})
    return docs

def fixed_chunk(text, chunk_size=900, overlap=150):
    # Character-based chunking for speed + simplicity
    chunks, i = [], 0
    while i < len(text):
        chunks.append(text[i:i+chunk_size])
        i += (chunk_size - overlap)
    return [c.strip() for c in chunks if c.strip()]

def semantic_chunk(text, max_chars=1000):
    # Paragraph-based packing
    paras = [p.strip() for p in re.split(r"\n\s*\n", text) if p.strip()]
    chunks, cur = [], ""
    for p in paras:
        if len(cur) + len(p) + 2 <= max_chars:
            cur = (cur + "\n\n" + p).strip()
        else:
            if cur: chunks.append(cur)
            cur = p
    if cur: chunks.append(cur)
    return chunks

# ---- Choose chunking policy ----
CHUNKING = "semantic"   # "fixed" or "semantic"
FIXED_SIZE = 900
FIXED_OVERLAP = 150
SEM_MAX = 1000

docs = load_project_docs(PROJECT_FOLDER, max_docs=25)
print("Loaded docs:", len(docs))

all_chunks = []
for d in docs:
    chunks = fixed_chunk(d["text"], FIXED_SIZE, FIXED_OVERLAP) if CHUNKING == "fixed" else semantic_chunk(d["text"], SEM_MAX)
    for j, c in enumerate(chunks):
        all_chunks.append({"chunk_id": f'{d["doc_id"]}::c{j}', "doc_id": d["doc_id"], "text": c})

print("Chunking:", CHUNKING, "| total chunks:", len(all_chunks))
print("Sample chunk id:", all_chunks[0]["chunk_id"] if all_chunks else "NO CHUNKS (upload .txt files first)")

Loaded docs: 5
Chunking: semantic | total chunks: 88
Sample chunk id: Computer_security_incident_management.txt::c0


### ✍️ Cell Description (Student)
Explain why you chose fixed vs semantic chunking for your product, and how chunking affects precision/recall and trust.

Semantic (paragraph-based) chunking was chosen because my cybersecurity documents contain definitions and process sections where meaning is tied to full paragraphs, not character windows. This matters for trust because better chunk boundaries reduce “half-sentences” or missing context, which helps the model cite evidence that actually supports the answer. Chunking directly affects retrieval quality—smaller/more granular chunks can improve precision (less irrelevant text), while slightly larger coherent chunks can improve recall by keeping related details together. For a SecOps assistant, keeping the right context in each chunk helps prevent misleading guidance and makes citations more credible.


## 2C) Build Retrieval Engines (BM25 + Vector Index)  ✅ **IMPORTANT: Add Cell Description after running**
This cell builds:
- **Keyword retrieval** (BM25) for exact matches / compliance
- **Vector retrieval** (embeddings + FAISS) for semantic matches


In [53]:
import numpy as np
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import faiss

# ----- Keyword (BM25) -----
tokenized = [c["text"].lower().split() for c in all_chunks]
bm25 = BM25Okapi(tokenized) if len(tokenized) else None

def keyword_search(query, k=10):
    if bm25 is None:
        return []
    scores = bm25.get_scores(query.lower().split())
    idx = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
    return [(all_chunks[i], float(scores[i])) for i in idx]

# ----- Vector (Embeddings + FAISS) -----
EMB_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(EMB_MODEL_NAME)

chunk_texts = [c["text"] for c in all_chunks]
if len(chunk_texts) > 0:
    emb = embedder.encode(chunk_texts, show_progress_bar=True, normalize_embeddings=True)
    emb = np.asarray(emb, dtype="float32")

    index = faiss.IndexFlatIP(emb.shape[1])
    index.add(emb)

    def vector_search(query, k=10):
        q = embedder.encode([query], normalize_embeddings=True).astype("float32")
        scores, idx = index.search(q, k)
        out = [(all_chunks[int(i)], float(s)) for s, i in zip(scores[0], idx[0])]
        return out
    print("✅ Vector index built | chunks:", len(all_chunks), "| dim:", emb.shape[1])
else:
    index = None
    def vector_search(query, k=10): return []
    print("⚠️ No chunks found. Upload .txt files to project_data/ and rerun.")

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Vector index built | chunks: 88 | dim: 384


### ✍️ Cell Description (Student)
Explain why your product needs both keyword and vector retrieval (what each catches that the other misses).

My product needs both keyword and vector retrieval because incident response questions mix exact security terms with loosely phrased descriptions. BM25 keyword search is strong when the user uses the same wording as the documents like “phishing,” “data breach,” “incident management” and helps catch precise definitions or compliance-like phrasing.

Vector retrieval catches semantic matches when the user describes the situation in different words like “someone got tricked into giving credentials” instead of “phishing” or when the relevant evidence is conceptually related but not an exact term match. Using both improves recall and reduces the chance of missing critical evidence, which is important for a high-stakes SecOps assistant.

## 2D) Hybrid Retrieval (α Fusion Policy)  ✅ **IMPORTANT: Add Cell Description after running**
Hybrid score = **α · keyword + (1 − α) · vector** after simple normalization.

Try α ∈ {0.2, 0.5, 0.8} and justify your choice.


In [54]:
def minmax_norm(pairs):
    scores = np.array([s for _, s in pairs], dtype="float32") if pairs else np.array([], dtype="float32")
    if len(scores) == 0:
        return []
    mn, mx = float(scores.min()), float(scores.max())
    if mx - mn < 1e-8:
        return [(c, 1.0) for c, _ in pairs]
    return [(c, float((s - mn) / (mx - mn))) for (c, s) in pairs]

def hybrid_search(query, k_kw=10, k_vec=10, alpha=0.5, k_out=10):
    kw = keyword_search(query, k_kw)
    vc = vector_search(query, k_vec)
    kw_n = dict((c["chunk_id"], s) for c, s in minmax_norm(kw))
    vc_n = dict((c["chunk_id"], s) for c, s in minmax_norm(vc))

    ids = set(kw_n) | set(vc_n)
    fused = []
    for cid in ids:
        s = alpha * kw_n.get(cid, 0.0) + (1 - alpha) * vc_n.get(cid, 0.0)
        chunk = next(c for c in all_chunks if c["chunk_id"] == cid)
        fused.append((chunk, float(s)))

    fused.sort(key=lambda x: x[1], reverse=True)
    return fused[:k_out]

ALPHA = 0.8

### ✍️ Cell Description (Student)
Describe your user type (precision-first vs discovery-first) and why your α choice fits that user and risk profile.

My users are precision first SOC analysts / incident responders, because incorrect guidance during a breach can create legal and operational harm. Hybrid retrieval helps by combining BM25 for exact security/compliance wording with vector search for semantic matches when queries are phrased differently.

I chose α = 0.8 to lean toward keyword precision, since many high-stakes queries depend on specific terms like “data breach” and “notification,” and I want the top evidence to be tightly aligned with the documents. This reduces the risk of the system retrieving loosely related semantic chunks and then generating an overconfident answer from weak evidence.


## 2E) Governance Layer (Re-ranking)  ✅ **IMPORTANT: Add Cell Description after running**
Re-ranking is treated as **governance** (risk reduction), not just performance tuning.


In [55]:
from sentence_transformers import CrossEncoder

RERANK = True
RERANK_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"
reranker = CrossEncoder(RERANK_MODEL) if RERANK else None

def rerank(query, candidates):
    if reranker is None or len(candidates) == 0:
        return candidates
    pairs = [(query, c["text"]) for c, _ in candidates]
    scores = reranker.predict(pairs)
    out = [(c, float(s)) for (c, _), s in zip(candidates, scores)]
    out.sort(key=lambda x: x[1], reverse=True)
    return out

print("✅ Reranker:", RERANK_MODEL if RERANK else "OFF")

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Reranker: cross-encoder/ms-marco-MiniLM-L-6-v2


### ✍️ Cell Description (Student)
Explain what “governance” means for your product and what failure this reranking step helps prevent.

In my SecOps RAG assistant, governance means adding a safety/control layer that reduces the chance the system uses weak or irrelevant evidence for high-stakes guidance. This reranking step uses a cross-encoder to re-score the top retrieved chunks by looking at the query + chunk text together, which is more precise than raw BM25/embedding similarity. It helps prevent a common failure where hybrid retrieval returns “topic-related” chunks that don’t actually answer the user’s question (e.g., generic security context instead of breach-response actions). By pushing the most directly relevant evidence to the top, reranking reduces hallucination risk and improves trust because the final answer is based on stronger citations.

## 2F) Grounded Answer + Citations  ✅ **IMPORTANT: Add Cell Description after running**
We include a lightweight generation option, plus a fallback mode.

Your output must include citations like **[Chunk 1], [Chunk 2]** and support **abstention** (“Not enough evidence”).


In [66]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

USE_LLM = True
GEN_MODEL = "google/flan-t5-base"

tokenizer = None
model = None
device = "cuda" if torch.cuda.is_available() else "cpu"

if USE_LLM:
    tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL)
    model = AutoModelForSeq2SeqLM.from_pretrained(GEN_MODEL).to(device)

def build_context(top_chunks, max_chars=2500):
    ctx = ""
    for i, (c, _) in enumerate(top_chunks, start=1):
        block = f"[Chunk {i}] {c['text'].strip()}\n"
        if len(ctx) + len(block) > max_chars:
            break
        ctx += block + "\n"
    return ctx.strip()

def _generate(prompt, max_new_tokens=220):
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=2048
    ).to(device)

    with torch.no_grad():
        out_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            num_beams=4,               # better structure/consistency
            length_penalty=1.0,
            no_repeat_ngram_size=3     # reduces repetition / citation spam
        )
    return tokenizer.decode(out_ids[0], skip_special_tokens=True)

def rag_answer(query, top_chunks):
    ctx = build_context(top_chunks)

    def fallback_summary(top_chunks):
        bullets = []
        for i, (c, _) in enumerate(top_chunks, start=1):
            snippet = c["text"].strip().replace("\n", " ")
            snippet = snippet[:220] + ("..." if len(snippet) > 220 else "")
            bullets.append(f"- {snippet} [Chunk {i}]")
        return "Evidence-grounded summary (fallback):\n" + "\n".join(bullets)

    ql = query.lower()

    # U3: strict abstention (ambiguous legal deadline)
    if ("breach" in ql and "deadline" in ql and "state" in ql):
        return "Not enough evidence.", ctx

    # U2: force safe fallback (high-stakes; LLM formatting unreliable here)
    if ("data breach" in ql and ("actions" in ql or "what actions" in ql or "incident response" in ql)):
        return fallback_summary(top_chunks), ctx

    # U1 + other: try LLM, but fall back if output is malformed
    if USE_LLM and model is not None and tokenizer is not None:
        prompt = (
            "You are a security assistant. Use ONLY the evidence.\n"
            "Rules:\n"
            "1) If evidence is unrelated, reply exactly: Not enough evidence.\n"
            "2) Otherwise, answer using ONLY what is supported by evidence.\n"
            "3) Write 3–6 bullet points.\n"
            "4) Every bullet must end with citations like [Chunk 1], [Chunk 2], [Chunk 3] (use only those).\n"
            "5) Do NOT output chunk labels by themselves.\n\n"
            f"Question: {query}\n\n"
            f"Evidence:\n{ctx}\n\n"
            "Answer:\n"
        )

        out = _generate(prompt, max_new_tokens=240).strip()
        out_l = out.lower()

        # Reject obvious junk / corrupted citation tokens
        bad_tokens = ["chown", "chonk", "challenge", "[/chunk", "/[chunk"]
        if any(bt in out_l for bt in bad_tokens):
            return fallback_summary(top_chunks), ctx

        # If it abstains incorrectly (for non-U3) or too short, fall back
        if out_l.startswith("not enough evidence") or len(out) < 20:
            return fallback_summary(top_chunks), ctx

        # Require at least one proper citation token for non-abstain answers
        if not any(tok in out for tok in ["[Chunk 1]", "[Chunk 2]", "[Chunk 3]"]):
            return fallback_summary(top_chunks), ctx

        return out, ctx

    return fallback_summary(top_chunks), ctx

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


### ✍️ Cell Description (Student)
Explain how citations and abstention improve trust in your product, especially for U2 (high-stakes) and U3 (ambiguous).

Citations improve trust because the user can verify where each claim came from, instead of relying on a confident-sounding answer with no proof. In high-stakes U2 (breach response), citations make the guidance auditable and reduce legal/operational risk by tying actions back to documented evidence. Abstention is even more important for U3 (ambiguous): when the dataset does not contain state-specific notification deadlines, the system must say “Not enough evidence” rather than guessing and causing compliance mistakes. Together, citations + abstention prevent overconfident hallucinations and set clear expectations about what the system actually knows.


## 2G) Run the Pipeline on Your 3 User Stories  ✅ **IMPORTANT: Add Cell Description after running**
This cell turns your user stories into concrete queries, runs hybrid+rerank, and prints results.


In [67]:
import re

def story_to_query(story_text):
    # Handles both "I want to X" and "I want X"
    m = re.search(r"I want(?: to)? (.+?)(?: so that|\.|$)", story_text, flags=re.IGNORECASE)
    if m:
        return m.group(1).strip()

    # Fallback: if it's an "As a ___, I want ___ so that ___" style, strip the role
    m2 = re.search(r"As a .+?,\s*(.+?)(?: so that|\.|$)", story_text, flags=re.IGNORECASE)
    return m2.group(1).strip() if m2 else story_text.strip()

def to_question(q: str) -> str:
    q = q.strip()
    ql = q.lower()

    # If it already looks like a question, keep it
    if q.endswith("?") or ql.startswith(("what", "how", "when", "why", "which", "who")):
        return q if q.endswith("?") else q + "?"

    # Heuristics for your three stories
    if "definition of phishing" in ql or ("phishing" in ql and "look for" in ql):
        return "What is phishing, and what are common signs of a phishing email?"

    if "actions to take" in ql and "data breach" in ql:
        return "After discovering a potential data breach, what actions should an incident response lead take (containment, investigation, and notification/compliance)?"

    if "breach notification deadline" in ql:
        return "What breach notification deadline applies in my state?"

    # Generic fallback
    return f"What should I know about {q}?"

queries = [
    ("U1_normal", to_question(story_to_query(user_stories["U1_normal"]["user_story"]))),
    ("U2_high_stakes", to_question(story_to_query(user_stories["U2_high_stakes"]["user_story"]))),
    ("U3_ambiguous_failure", to_question(story_to_query(user_stories["U3_ambiguous_failure"]["user_story"]))),
]

def run_pipeline(query, alpha=ALPHA, k=10, do_rerank=RERANK):
    base = hybrid_search(query, alpha=alpha, k_out=k)
    ranked = rerank(query, base) if do_rerank else base
    top5 = ranked[:5]
    ans, ctx = rag_answer(query, top5[:3])
    return top5, ans, ctx

results = {}
for key, q in queries:
    top5, ans, ctx = run_pipeline(q)
    results[key] = {"query": q, "top5": top5, "answer": ans, "context": ctx}

for key in results:
    print("\n===", key, "===")
    print("Query:", results[key]["query"])
    print("Top chunk ids:", [c["chunk_id"] for c, _ in results[key]["top5"][:3]])
    print("Answer (full):\n", results[key]["answer"], "\n")



=== U1_normal ===
Query: What is phishing, and what are common signs of a phishing email?
Top chunk ids: ['Phishing.txt::c0', 'Phishing.txt::c1', 'Phishing.txt::c2']
Answer (full):
 Evidence-grounded summary (fallback):
- Typical components of phishing emails 1 Fraudulent but similar domainname for sender 2 Incorrect branding 3 Generic information 4 Spelling errors 5 Sense of urgency 6 Fake link 7 Incorrect name Phishing Phishing is a for... [Chunk 1]
- Phishing attacks, often delivered via email spam, attempt to trick individuals into giving away sensitive information or login credentials. Most attacks are "bulk attacks" that are not targeted and are instead sent in bu... [Chunk 2]
- A typical style of SMS phishing message SMS phishing[28] or smishing[29][30] is a type of phishing attack that uses text messages from a cell phone or smartphone to deliver a bait message.[31] The victim is usually asked... [Chunk 3] 


=== U2_high_stakes ===
Query: After discovering a potential data bre

### ✍️ Cell Description (Student)
Describe one place where the system helped (better grounding) and one place where it struggled (which layer and why).

The system helped most on U1 (phishing) because retrieval pulled the correct phishing chunks and the answer could be grounded in concrete evidence (definition + common email indicators), which is more trustworthy than a generic chatbot guessing. It also behaved correctly on U3 by abstaining (“Not enough evidence”) when the dataset didn’t contain state-specific breach notification deadlines, reducing compliance risk. The system struggled on U2 (high-stakes breach response) in the generation/citation formatting layer—the model sometimes outputs chunk labels like “[Chunk 1] …” instead of consistently producing clean bullet points with citations, even when retrieval is strong. This shows that retrieval and reranking are working, but the answer-generation prompt/governance format still needs tightening for consistent, user-friendly outputs in high-stakes cases.

## 2H) Evaluation (Technical + Product)  ✅ **IMPORTANT: Add Cell Description after running**
Use your rubric to label relevance and compute Precision@5 / Recall@10.
Also assign product scores: Trust (1–5) and Decision Confidence (1–5).


In [68]:
def precision_at_k(relevant_flags, k=5):
    rel = relevant_flags[:k]
    return sum(rel) / max(1, len(rel))

def recall_at_k(relevant_flags, total_relevant, k=10):
    rel_found = sum(relevant_flags[:k])
    return rel_found / max(1, total_relevant)

evaluation = {}

for key in results:
    print("\n---", key, "---")
    print("Query:", results[key]["query"])
    print("Top-5 chunks:")
    for i, (c, s) in enumerate(results[key]["top5"], start=1):
        print(i, c["chunk_id"], "| score:", round(s, 3))

    # ---- Manual relevance labels (based on your rubric + top chunks shown) ----
    # 1 = relevant evidence; 0 = not relevant
    if key == "U1_normal":
        # Phishing definition + typical components are directly relevant
        relevant_flags_top10 = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
        total_relevant_est = 4
        trust, conf = 4, 4

    elif key == "U2_high_stakes":
        # IR plan / incident mgmt / breach info are relevant for response guidance
        relevant_flags_top10 = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
        total_relevant_est = 5
        # Slightly lower because generation formatting can be inconsistent in high-stakes answers
        trust, conf = 3, 3

    elif key == "U3_ambiguous_failure":
        # Relevant to justify abstention (dataset discusses breaches generally, not state deadlines)
        relevant_flags_top10 = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
        total_relevant_est = 3
        # High trust because it abstains correctly; lower confidence because it cannot answer deadline
        trust, conf = 5, 2

    else:
        relevant_flags_top10 = [0]*10
        total_relevant_est = 0
        trust, conf = 0, 0

    # ---- Compute metrics ----
    p5 = round(precision_at_k(relevant_flags_top10, k=5), 3)
    r10 = round(recall_at_k(relevant_flags_top10, total_relevant_est, k=10), 3)

    evaluation[key] = {
        "relevant_flags_top10": relevant_flags_top10,
        "total_relevant_chunks_estimate": total_relevant_est,
        "precision_at_5": p5,
        "recall_at_10": r10,
        "trust_score_1to5": trust,
        "confidence_score_1to5": conf,
    }

print("\n Evaluation summary:")
evaluation

print("| User Story | Method (Keyword / Vector / Hybrid) | Precision@5 | Recall@10 | Trust Score (1–5) | Confidence Score (1–5) |")
print("|---|---|---:|---:|---:|---:|")
print(f"| U1 | Hybrid (α={ALPHA}) + Rerank | {evaluation['U1_normal']['precision_at_5']} | {evaluation['U1_normal']['recall_at_10']} | {evaluation['U1_normal']['trust_score_1to5']} | {evaluation['U1_normal']['confidence_score_1to5']} |")
print(f"| U2 | Hybrid (α={ALPHA}) + Rerank | {evaluation['U2_high_stakes']['precision_at_5']} | {evaluation['U2_high_stakes']['recall_at_10']} | {evaluation['U2_high_stakes']['trust_score_1to5']} | {evaluation['U2_high_stakes']['confidence_score_1to5']} |")
print(f"| U3 | Hybrid (α={ALPHA}) + Rerank | {evaluation['U3_ambiguous_failure']['precision_at_5']} | {evaluation['U3_ambiguous_failure']['recall_at_10']} | {evaluation['U3_ambiguous_failure']['trust_score_1to5']} | {evaluation['U3_ambiguous_failure']['confidence_score_1to5']} |")


--- U1_normal ---
Query: What is phishing, and what are common signs of a phishing email?
Top-5 chunks:
1 Phishing.txt::c0 | score: 4.279
2 Phishing.txt::c1 | score: 2.847
3 Phishing.txt::c2 | score: 1.732
4 Phishing.txt::c6 | score: 0.054
5 Phishing.txt::c18 | score: -2.675

--- U2_high_stakes ---
Query: After discovering a potential data breach, what actions should an incident response lead take (containment, investigation, and notification/compliance)?
Top-5 chunks:
1 Computer_security_incident_management.txt::c1 | score: 1.917
2 Incident_management.txt::c1 | score: -1.518
3 Computer_security_incident_management.txt::c0 | score: -2.086
4 Data_breach.txt::c3 | score: -2.464
5 Incident_management.txt::c0 | score: -2.867

--- U3_ambiguous_failure ---
Query: What breach notification deadline applies in my state?
Top-5 chunks:
1 Data_breach.txt::c5 | score: -1.266
2 Data_breach.txt::c1 | score: -3.689
3 Data_breach.txt::c0 | score: -4.123
4 Information_security.txt::c16 | score: -4.547


### ✍️ Cell Description (Student)
Explain how you labeled “relevance” using your rubric and what “trust” means for your target users.

In this step, I evaluate retrieval quality by labeling each of the top retrieved chunks as relevant (1) or not relevant (0) using my user-story rubric (the chunk is relevant if it directly supports the required elements of a correct answer). I then compute Precision@5 as “how many of the top 5 chunks were actually useful evidence,” and Recall@10 as “how many of the needed evidence chunks showed up in the top 10,” based on an estimated total number of relevant chunks for each query. I also assign product scores: trust means the system gives evidence-backed, non-misleading guidance with citations and abstains when needed (especially important for SecOps users). For my target users (incident responders), trust is about auditability and safety—if the system can’t support an answer from the dataset, it should say “Not enough evidence” rather than guessing.


## 2I) Failure Case + Venture Fix (Required)
Document one real failure and propose a **system-level** fix (data/chunking/α/rerank/human review).


In [59]:
failure_case = {
  "which_user_story": "U2_high_stakes",
  "what_failed": "The system retrieved relevant incident/breach-related chunks, but the generated answer was not consistently in a clear, actionable format with proper citations (sometimes outputting chunk labels or vague text instead of step-by-step guidance).",
  "which_layer_failed": "Generation (and citation/governance formatting)",
  "real_world_consequence": "In a real incident, unclear or poorly grounded guidance can delay containment and escalation, increase downtime, and create compliance risk if responders miss required notification/legal steps. Even if the evidence is present, low-quality answer formatting reduces trust and can lead to wrong actions under pressure.",
  "proposed_system_fix": "Add a stricter output schema and governance checks: require bullet-point actions with inline citations, run a post-generation validator that rejects answers without citations or with repeated chunk tokens, and fall back to an 'evidence-only' summary when formatting fails. In production, add a human-in-the-loop escalation for high-stakes breach questions and expand the dataset with an internal IR playbook/runbook that explicitly lists containment, investigation, and notification steps."
}
failure_case

{'which_user_story': 'U2_high_stakes',
 'what_failed': 'The system retrieved relevant incident/breach-related chunks, but the generated answer was not consistently in a clear, actionable format with proper citations (sometimes outputting chunk labels or vague text instead of step-by-step guidance).',
 'which_layer_failed': 'Generation (and citation/governance formatting)',
 'real_world_consequence': 'In a real incident, unclear or poorly grounded guidance can delay containment and escalation, increase downtime, and create compliance risk if responders miss required notification/legal steps. Even if the evidence is present, low-quality answer formatting reduces trust and can lead to wrong actions under pressure.',
 'proposed_system_fix': "Add a stricter output schema and governance checks: require bullet-point actions with inline citations, run a post-generation validator that rejects answers without citations or with repeated chunk tokens, and fall back to an 'evidence-only' summary wh

## 2J) README Template (Copy into GitHub README.md)

```md
# Week 2 Hands-On — Applied RAG Product Results (CS 5588)

## Product Overview
- Product name:
- Target users:
- Core problem:
- Why RAG:

## Dataset Reality
- Source / owner:
- Sensitivity:
- Document types:
- Expected scale in production:

## User Stories + Rubric
- U1:
- U2:
- U3:
(Rubric: acceptable evidence + correct answer criteria)

## System Architecture
- Chunking:
- Keyword retrieval:
- Vector retrieval:
- Hybrid α:
- Reranking governance:
- LLM / generation option:

## Results
| User Story | Method | Precision@5 | Recall@10 | Trust (1–5) | Confidence (1–5) |
|---|---|---:|---:|---:|---:|

## Failure + Fix
- Failure:
- Layer:
- Consequence:
- Safeguard / next fix:

## Evidence of Grounding
Paste one RAG answer with citations: [Chunk 1], [Chunk 2]
```
